<a href="https://colab.research.google.com/github/helenross636/magic_season/blob/main/%23Task17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import time
from collections import defaultdict

In [47]:
# Устанавливаем seed для повторяемости результатов
np.random.seed(42)
# Загружаем данные
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Преобразование размерности изображений
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [48]:
#random.seed(42)
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/seed(42)/model1.h5')
model2 = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/seed(42)/model2.h5')
model3 = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/seed(42)/model3.h5')
model4 = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/seed(42)/model4.h5')
w5000 = (model3.layers[3].kernel)
print(w5000.shape)

(5000, 10)


In [49]:
models = []
models.append(model)
models.append(model2)
models.append(model3)
models.append(model4)

# #TASK 17

In [58]:
w1500 = (models[1].layers[3].kernel)
print(w1500.shape)
w5000 = (models[2].layers[3].kernel)
print(w5000.shape)

w_PLUS = np.vstack((w1500,w5000))
print('Конкатенация: ',w_PLUS.shape)

(y,z) = w_PLUS.shape
am = np.zeros((y,y))
am = abs(np.corrcoef(w_PLUS))
np.fill_diagonal(am, 0)
print('матрица по метрике корелляции: ',am.shape)

am2 = am[1500:,0:1500]
print('матрица [1500:,0:1500] по метрике корелляции: ',am2.shape)
mask = am2 > 0.9
print('Проверка размерности маски: ', mask.shape)

#similar = {} #представляют тип данных словаря dict, а словарь состоит из групп значений пар ключей
similar = defaultdict(list)
count = 0;  #сколько нейронов надо будет удалять из сети 5к
for idx, k in enumerate(mask):
  nk = k.nonzero()[0]
  if len(nk) != 0:
    similar[nk[np.argmax(am2[idx, nk])]].append(idx)
    count+=1

print('Длина пар ключей (> 0.9): ',len(similar))


(1500, 10)
(5000, 10)
Конкатенация:  (6500, 10)
матрица по метрике корелляции:  (6500, 6500)
матрица [1500:,0:1500] по метрике корелляции:  (5000, 1500)
Проверка размерности маски:  (5000, 1500)
Длина пар ключей (> 0.9):  1175


In [51]:
print('len similar',len(similar))

len similar 1175


In [52]:
KEY_S = list(similar.keys())
VALUE_S = list(similar.values())

print(KEY_S) #keys
print('len(KEY_S)',len(KEY_S)) #keys

#print(VALUE_S) #words
similar_copy = similar

[429, 956, 113, 1009, 901, 186, 230, 845, 711, 856, 1229, 1286, 1279, 1231, 1168, 11, 547, 568, 820, 1420, 627, 407, 1314, 1319, 516, 1214, 967, 82, 319, 662, 344, 1445, 940, 320, 1246, 1120, 359, 577, 945, 1031, 105, 534, 325, 384, 153, 754, 1155, 784, 1102, 802, 847, 696, 691, 632, 486, 638, 296, 1220, 174, 561, 292, 1135, 324, 529, 299, 460, 1060, 770, 1141, 591, 1455, 935, 611, 386, 1096, 421, 633, 1111, 416, 401, 436, 682, 1373, 1410, 1331, 256, 877, 1358, 1498, 411, 439, 692, 1332, 1416, 1251, 1019, 748, 1315, 70, 220, 826, 371, 843, 1076, 1042, 858, 497, 1040, 1462, 454, 860, 906, 790, 723, 689, 717, 317, 652, 368, 1475, 339, 1484, 1449, 109, 1079, 962, 476, 290, 589, 1219, 555, 1032, 880, 1119, 1088, 258, 523, 1039, 59, 1359, 849, 828, 526, 893, 302, 388, 141, 744, 1428, 124, 393, 176, 988, 387, 678, 1132, 923, 281, 58, 115, 1444, 71, 178, 392, 939, 1268, 816, 1281, 309, 585, 338, 653, 1244, 594, 541, 1483, 316, 190, 944, 322, 999, 1292, 517, 801, 23, 62, 737, 1125, 29, 918, 12

In [53]:
print('len similar',len(similar))

len similar 1175


In [54]:
del_keys = []  #ключи (слабых нейронов)


for idx, value in enumerate(KEY_S): #ходим по значениям списка ключей
  for idx0, value0 in enumerate(VALUE_S): #ходим во всем списакм слов
    if(value in value0):  #сравнимаем ключи со всеми списками слов и если есть СОВПАДЕНИЕ
      if( (len(similar_copy[idx])) < (len(similar_copy[idx0])) ): #сравниваем длину
        del_keys.append(value)  #записываем значение слабого ключа

print('СПИСОК СЛАБЫХ КЛЮЧЕЙ:',del_keys)
print('LEN',len(del_keys))


СПИСОК СЛАБЫХ КЛЮЧЕЙ: [901, 230, 856, 11, 407, 1445, 1031, 105, 153, 1155, 802, 696, 632, 561, 770, 1141, 1455, 1410, 439, 826, 1076, 1040, 1462, 454, 790, 652, 368, 1484, 109, 1032, 302, 141, 744, 923, 115, 1268, 338, 322, 999, 1292, 1125, 918, 1215, 194, 937, 167, 47, 829, 838, 663, 1233, 1037, 575, 853, 1134, 1077, 151, 870, 808, 1310, 137, 752, 1464, 932, 1346, 1401, 1260, 64, 624, 887, 458, 1028, 612, 336, 865, 811, 255, 554, 310, 1232, 1285, 1166, 1426, 198, 602, 1350, 1078, 753, 48, 1082, 1284, 376, 1316, 117, 356, 254, 83, 148, 1142, 1066, 979, 704, 41, 242, 1303, 427, 905, 1461, 1007, 84, 731, 1329, 1496, 36, 495, 251, 544, 1380, 1051, 354, 321, 917, 832, 1276, 104, 247, 330, 1457, 590, 992, 274, 841, 715, 450, 1154, 1036, 757, 514, 1024, 1205, 357, 928, 233, 625, 498, 1419, 1274, 1295, 249, 827, 428, 1115, 761, 1434, 445, 588, 579, 1234, 342, 398, 695, 211, 415, 99, 701, 315, 1140, 337, 136, 306, 694, 871, 446, 1103, 1086, 278, 191, 1018, 101, 145, 1002, 882, 607, 519, 1117, 

In [59]:
#удаляем из списка слабые нейроны
similar_pop = similar.copy()

for idx, value in enumerate(del_keys):
  similar_pop.pop(value, None)

print('len similar',len(similar))
print('len similar_pop',len(similar_pop))

len similar 1175
len similar_pop 914


***ПОЛУЧИЛИ СПИСОК ТОЛЬКО СИЛЬНЫХ КЛЮЧЕЙ***

In [60]:
new_similar = similar_pop
print('len(new_similar)',len(new_similar))

len(new_similar) 914


# ***Спартанский подход:***

In [61]:
nu = list(new_similar.keys())
print(nu)
print(len(nu))

[429, 956, 113, 1009, 186, 845, 711, 1229, 1286, 1279, 1231, 1168, 547, 568, 820, 1420, 627, 1314, 1319, 516, 1214, 967, 82, 319, 662, 344, 940, 320, 1246, 1120, 359, 577, 945, 534, 325, 384, 754, 784, 1102, 847, 691, 486, 638, 296, 1220, 174, 292, 1135, 324, 529, 299, 460, 1060, 591, 935, 611, 386, 1096, 421, 633, 1111, 416, 401, 436, 682, 1373, 1331, 256, 877, 1358, 1498, 411, 692, 1332, 1416, 1251, 1019, 748, 1315, 70, 220, 371, 843, 1042, 858, 497, 860, 906, 723, 689, 717, 317, 1475, 339, 1449, 1079, 962, 476, 290, 589, 1219, 555, 880, 1119, 1088, 258, 523, 1039, 59, 1359, 849, 828, 526, 893, 388, 1428, 124, 393, 176, 988, 387, 678, 1132, 281, 58, 1444, 71, 178, 392, 939, 816, 1281, 309, 585, 653, 1244, 594, 541, 1483, 316, 190, 944, 517, 801, 23, 62, 737, 29, 1223, 508, 643, 1396, 157, 74, 656, 1452, 610, 767, 94, 548, 938, 1201, 348, 480, 840, 1278, 814, 471, 1289, 603, 978, 980, 1326, 147, 750, 1377, 566, 697, 742, 161, 1194, 203, 505, 365, 1411, 1459, 364, 998, 573, 252, 385, 1

In [62]:
count = np.asarray([len(x) for x in new_similar.values()])
count = count[..., np.newaxis]
count = np.repeat(count, 10, axis=1)
count.shape

(914, 10)

In [63]:
wi2 = models[2].layers[2].kernel.numpy()[:, list(nu)]  #вход из 5000 (keras)
bi2 = models[2].layers[2].bias.numpy()[list(nu)]  #вход из 5000 (bias)
print('wi2',wi2.shape)
print('bi2',bi2.shape)

wo2 = models[2].layers[3].kernel.numpy()[list(new_similar.keys())] * count
bo2 = (models[2].layers[3].bias)
print('wo2',wo2.shape)
print('bo2',bo2.shape)


wi2 (784, 914)
bi2 (914,)
wo2 (914, 10)
bo2 (10,)


In [64]:
#создаем модель
inputs = layers.Input(shape = (784,), name = "in_layer")
x = layers.Flatten()(inputs)
x = layers.Dense(len(nu), activation="relu", name = "hidden_layer")(x)
outputs = layers.Dense(10, activation="softmax", name = "out_layer")(x)
model_Colab = tf.keras.Model(inputs = inputs, outputs = outputs)
model_Colab.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model_Colab.summary())

model_Colab.layers[2].set_weights([wi2, bi2])
model_Colab.layers[3].set_weights([wo2, bo2])

#смотрим результат
scores = model_Colab.evaluate(X_test, Y_test, verbose=0);
print("СПАРТАНСКАЯ МОДЕЛЬ. Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))  

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_layer (InputLayer)       [(None, 784)]             0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 hidden_layer (Dense)        (None, 914)               717490    
                                                                 
 out_layer (Dense)           (None, 10)                9150      
                                                                 
Total params: 726,640
Trainable params: 726,640
Non-trainable params: 0
_________________________________________________________________
None
СПАРТАНСКАЯ МОДЕЛЬ. Точность работы на тестовых данных: 94.14%


**ТЕПЕРЬ ДЛЯ МОДЕЛЕЙ 1500, 5000 И 10000**
(ДО ЭТОГО СЛОВАРЬ СТРОИЛСЯ ИСХОДЯ ИЗ КОНКАТЕНАЦИИ)

In [92]:
simils = []
model_num = [1,2,3]

for i in list(model_num):
  print('MODEL NUMBER',i+1)
  weight = (models[i].layers[3].kernel)
  print(weight.shape)

  (y,z) = weight.shape
  am = np.zeros((y,y))
  am = abs(np.corrcoef(weight))
  np.fill_diagonal(am, 0)
  print('матрица по метрике корелляции: ',am.shape)

  mask = am > 0.9
  print('Проверка размерности маски: ', mask.shape)

  #similar = {} #представляют тип данных словаря dict, а словарь состоит из групп значений пар ключей
  sim = defaultdict(list)
  count = 0;  #сколько нейронов надо будет удалять из сети 5к
  for idx, k in enumerate(mask):
    nk = k.nonzero()[0]
    if len(nk) != 0:
      sim[nk[np.argmax(am[idx, nk])]].append(idx)
      count+=1

  print('Длина пар ключей (> 0.9): ',len(sim))
  simils.append(sim)

  del_keys = []  #ключи (слабых нейронов)
  print('-------------------------------------------------')
  print('-------------------------------------------------')


MODEL NUMBER 2
(1500, 10)
матрица по метрике корелляции:  (1500, 1500)
Проверка размерности маски:  (1500, 1500)
Длина пар ключей (> 0.9):  642
-------------------------------------------------
-------------------------------------------------
MODEL NUMBER 3
(5000, 10)
матрица по метрике корелляции:  (5000, 5000)
Проверка размерности маски:  (5000, 5000)
Длина пар ключей (> 0.9):  2949
-------------------------------------------------
-------------------------------------------------
MODEL NUMBER 4
(10000, 10)
матрица по метрике корелляции:  (10000, 10000)
Проверка размерности маски:  (10000, 10000)
Длина пар ключей (> 0.9):  6368
-------------------------------------------------
-------------------------------------------------


In [106]:
del_ks = []
simil_pops = []

for i in range(len(simils)):
  print('MODEL NUMBER',model_num[i]+1)

  KEY_S = list(simils[i].keys())
  VALUE_S = list(simils[i].values())

  #print(KEY_S) #keys
  print('len(KEY_S)',len(KEY_S)) #keys

  #print(VALUE_S) #words
  similar_copy = simils[i].copy()
  print('similar_copy len',len(similar_copy))


  del_k = []  #ключи (слабых нейронов)
  for idx, value in enumerate(KEY_S): #ходим по значениям списка ключей
    for idx0, value0 in enumerate(VALUE_S): #ходим во всем списакм слов
      if(value in value0):  #сравнимаем ключи со всеми списками слов и если есть СОВПАДЕНИЕ
        if( (len(similar_copy[idx])) < (len(similar_copy[idx0])) ): #сравниваем длину
          del_k.append(value)  #записываем значение слабого ключа

  #print('СПИСОК СЛАБЫХ КЛЮЧЕЙ:',del_k)
  print('LEN',len(del_k))
  del_ks.append(del_k)
  similar_copy = simils[i].copy()
  print('similar_copy len (check)',len(similar_copy))

  #удаляем из списка слабые нейроны
  simil_pop = simils[i].copy()

  for idx, value in enumerate(del_k):
    simil_pop.pop(value, None)

  print('len simil',len(simils[i]))
  print('len simil_pop',len(simil_pop))
  simil_pops.append(simil_pop)

  print('-------------------------------------------------')
  print('-------------------------------------------------')


MODEL NUMBER 2
len(KEY_S) 642
similar_copy len 642
LEN 184
similar_copy len (check) 642
len simil 642
len simil_pop 458
-------------------------------------------------
-------------------------------------------------
MODEL NUMBER 3
len(KEY_S) 2949
similar_copy len 2949
LEN 984
similar_copy len (check) 2949
len simil 2949
len simil_pop 1965
-------------------------------------------------
-------------------------------------------------
MODEL NUMBER 4
len(KEY_S) 6368
similar_copy len 6368
LEN 2160
similar_copy len (check) 6368
len simil 6368
len simil_pop 4208
-------------------------------------------------
-------------------------------------------------


In [107]:
for i in range(len(simils)):
  print('MODEL NUMBER',model_num[i]+1)

  nu = list(simil_pops[i].keys())
  #print(nu)
  print(len(nu))

  count = np.asarray([len(x) for x in simil_pops[i].values()])
  count = count[..., np.newaxis]
  count = np.repeat(count, 10, axis=1)
  count.shape

  wi_new = models[model_num[i]].layers[2].kernel.numpy()[:, list(nu)]  #вход (keras)
  bi_new = models[model_num[i]].layers[2].bias.numpy()[list(nu)]  #вход (bias)
  print('wi2',wi_new.shape)
  print('bi2',bi_new.shape)

  wo_new = models[model_num[i]].layers[3].kernel.numpy()[list(simil_pops[i].keys())] * count
  bo_new = (models[model_num[i]].layers[3].bias)
  print('wo2',wo_new.shape)
  print('bo2',bo_new.shape)

  #создаем модель
  inputs = layers.Input(shape = (784,), name = "in_layer")
  x = layers.Flatten()(inputs)
  x = layers.Dense(len(nu), activation="relu", name = "hidden_layer")(x)
  outputs = layers.Dense(10, activation="softmax", name = "out_layer")(x)
  model_Sperta = tf.keras.Model(inputs = inputs, outputs = outputs)
  model_Sperta.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  print(model_Sperta.summary())

  model_Sperta.layers[2].set_weights([wi_new, bi_new])
  model_Sperta.layers[3].set_weights([wo_new, bo_new])

  #смотрим результат
  scores = model_Sperta.evaluate(X_test, Y_test, verbose=0);
  print("СПАРТАНСКАЯ МОДЕЛЬ. Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))  

  print('-------------------------------------------------')
  print('-------------------------------------------------')


MODEL NUMBER 2
458
wi2 (784, 458)
bi2 (458,)
wo2 (458, 10)
bo2 (10,)
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_layer (InputLayer)       [(None, 784)]             0         
                                                                 
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 hidden_layer (Dense)        (None, 458)               359530    
                                                                 
 out_layer (Dense)           (None, 10)                4590      
                                                                 
Total params: 364,120
Trainable params: 364,120
Non-trainable params: 0
_________________________________________________________________
None
СПАРТАНСКАЯ МОДЕЛЬ. Точность работы на тестовых данных: 95.42%
------------------------------------------------

# ***Гуманный подход***

In [97]:
u = {x for v in new_similar.values() for x in v}  #значения в словаре
q = {v for v in new_similar.keys()} #ключи в словаре
nu = set(range(5000)) - u - q

In [98]:
wi1 = models[2].layers[2].kernel.numpy()[:, list(nu)]  #вход из 5000 (keras)
bi1 = models[2].layers[2].bias.numpy()[list(nu)]  #вход из 1500 (bias)
print('wi1',wi1.shape)
print('bi1',bi1.shape)

wo1 = models[2].layers[3].kernel.numpy()[list(nu)]  #выход из 5000 (keras)
print('wo1',wo1.shape)
print('bo2',bo2.shape)

wi1 (784, 2131)
bi1 (2131,)
wo1 (2131, 10)
bo2 (10,)


In [ ]:
in_w = np.hstack((wi1, wi2)) #по столбцам
in_b = np.concatenate((bi1, bi2))  #по строкам
out_w = np.vstack((wo1, wo2))  #по строкам

print('in_w',in_w.shape)
print('in_b',in_b.shape)
print('out_w',out_w.shape)

(ox,oy)=out_w.shape

in_w (784, 3045)
in_b (3045,)
out_w (3045, 10)


In [ ]:
#создаем модель
inputs = layers.Input(shape = (784,), name = "in_layer")
x = layers.Flatten()(inputs)
x = layers.Dense(ox, activation="relu", name = "hidden_layer")(x)
outputs = layers.Dense(10, activation="softmax", name = "out_layer")(x)
model_G = tf.keras.Model(inputs = inputs, outputs = outputs)
model_G.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model_G.summary())

model_G.layers[2].set_weights([in_w, in_b])
model_G.layers[3].set_weights([out_w, bo2])

#смотрим результат
scores = model_G.evaluate(X_test, Y_test, verbose=0);
print("ГУМАННАЯ МОДЕЛЬ. Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))  

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_layer (InputLayer)       [(None, 784)]             0         
                                                                 
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 hidden_layer (Dense)        (None, 3045)              2390325   
                                                                 
 out_layer (Dense)           (None, 10)                30460     
                                                                 
Total params: 2,420,785
Trainable params: 2,420,785
Non-trainable params: 0
_________________________________________________________________
None
ГУМАННАЯ МОДЕЛЬ. Точность работы на тестовых данных: 97.37%


**ТЕПЕРЬ ДЛЯ МОДЕЛЕЙ 1500, 5000 И 10000**
(ДО ЭТОГО СЛОВАРЬ СТРОИЛСЯ ИСХОДЯ ИЗ КОНКАТЕНАЦИИ)

In [105]:
n_len = [1500,5000,10000]

for i in range(len(simils)):
  print('MODEL NUMBER',model_num[i]+1)

  u = {x for v in simil_pops[i].values() for x in v}  #значения в словаре
  q = {v for v in simil_pops[i].keys()} #ключи в словаре
  nu = set(range(n_len[i])) - u - q
  print('len(nu)',len(nu))

  wi11 = models[model_num[i]].layers[2].kernel.numpy()[:, list(nu)]  #вход (keras)
  bi11 = models[model_num[i]].layers[2].bias.numpy()[list(nu)]  #вход (bias)
  print('wi11',wi11.shape)
  print('bi11',bi11.shape)
  wo11 = models[model_num[i]].layers[3].kernel.numpy()[list(nu)]  #выход (keras)
  print('wo11',wo11.shape)

  nu = list(simil_pops[i].keys())
  print(len(nu))

  count = np.asarray([len(x) for x in simil_pops[i].values()])
  count = count[..., np.newaxis]
  count = np.repeat(count, 10, axis=1)
  count.shape

  wi22 = models[model_num[i]].layers[2].kernel.numpy()[:, list(nu)]  #вход (keras)
  bi22 = models[model_num[i]].layers[2].bias.numpy()[list(nu)]  #вход (bias)
  print('wi22',wi22.shape)
  print('bi22',bi22.shape)

  wo22 = models[model_num[i]].layers[3].kernel.numpy()[list(simil_pops[i].keys())] * count
  print('wo22',wo22.shape)

  in_w_new = np.hstack((wi11, wi22)) #по столбцам
  in_b_new = np.concatenate((bi11, bi22))  #по строкам
  out_w_new = np.vstack((wo11, wo22))  #по строкам

  print('in_w_new',in_w_new.shape)
  print('in_b_new',in_b_new.shape)
  print('out_w_new',out_w_new.shape)

  (ox,oy)=out_w_new.shape
  out_b_new = (models[model_num[i]].layers[3].bias)
  print('bo_new',out_b_new.shape)

  #создаем модель
  inputs = layers.Input(shape = (784,), name = "in_layer")
  x = layers.Flatten()(inputs)
  x = layers.Dense(ox, activation="relu", name = "hidden_layer")(x)
  outputs = layers.Dense(10, activation="softmax", name = "out_layer")(x)
  model_GUMAN = tf.keras.Model(inputs = inputs, outputs = outputs)
  model_GUMAN.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  print(model_GUMAN.summary())

  model_GUMAN.layers[2].set_weights([in_w_new, in_b_new])
  model_GUMAN.layers[3].set_weights([out_w_new, out_b_new])

  #смотрим результат
  scores = model_GUMAN.evaluate(X_test, Y_test, verbose=0);
  print("ГУМАННАЯ МОДЕЛЬ. Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

  print('-------------------------------------------------')
  print('-------------------------------------------------')

MODEL NUMBER 2
len(nu) 709
wi11 (784, 709)
bi11 (709,)
wo11 (709, 10)
458
wi22 (784, 458)
bi22 (458,)
wo22 (458, 10)
in_w_new (784, 1167)
in_b_new (1167,)
out_w_new (1167, 10)
bo_new (10,)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_layer (InputLayer)       [(None, 784)]             0         
                                                                 
 flatten_6 (Flatten)         (None, 784)               0         
                                                                 
 hidden_layer (Dense)        (None, 1167)              916095    
                                                                 
 out_layer (Dense)           (None, 10)                11680     
                                                                 
Total params: 927,775
Trainable params: 927,775
Non-trainable params: 0
______________________________________________________________

*просто на всякий случай сохраним*

Ранее строилась маска по матрице коэф корелляции > 0.9 (например). Потом начиналость рассмотрение каждой строки. Если в ней находились элементы с меткой `True`, то среди них искалось наибольшее значение в исходной матрице. И по индексу столбца `(nk)` исходной матрицы определялся ключ, в который записывался каждый индекс строки`(idx)`. В итоге, ключами становятся только наибольшие индексы столбцов `(nk[np.argmax(am[idx, nk])]`), а словами - все индексы строк `(idx)`. 
